<a href="https://colab.research.google.com/github/MaryamMortazaviSefat/new/blob/main/q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#load data
!wget -q https://raw.githubusercontent.com/MaryamMortazaviSefat/practice2_Prediction-of-Persian-Numbers/refs/heads/main/data_hoda.py
!wget -q https://github.com/MaryamMortazaviSefat/practice2_Prediction-of-Persian-Numbers/raw/refs/heads/main/dataset/Data_hoda_full.mat -P dataset

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from data_hoda import load_hoda
import random

In [4]:
x_train,y_train,x_test,y_test=load_hoda(size=25)

In [5]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [6]:
x_train = x_train.reshape(-1, 25, 25, 1)
x_test = x_test.reshape(-1, 25, 25, 1)

In [15]:
x_test.shape

(200, 25, 25, 1)

In [58]:
class ResidualBlock(layers.Layer):
    def __init__(self, filters, kernel_size=3, strides=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.conv2 = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')
        self.bn2 = layers.BatchNormalization()

        self.conv_shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')


    def call(self, x):
        shortcut = self.conv_shortcut(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        x += shortcut
        return self.relu(x)

In [85]:
class Cnn(Model):
  def __init__(self, num):
    super(Cnn, self).__init__()
    self.conv1 = layers.Conv2D(32, (3, 3), padding='same')
    self.bn1 = layers.BatchNormalization()
    self.res_block1 = ResidualBlock(64)
    self.res_block2 = ResidualBlock(128)
    self.res_block3 = ResidualBlock(200)
    self.pool = layers.MaxPooling2D(pool_size=(2, 2))
    self.flatten = layers.Flatten()
    self.fc = layers.Dense(num, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.pool(x)

    x = self.res_block1(x)
    x = self.pool(x)
    x = self.res_block2(x)
    x = self.pool(x)
    x = self.res_block3(x)
    x = self.pool(x)

    x = self.flatten(x)
    return self.fc(x)

num = 10
batch_size = 32
epochs = 30

model = Cnn(num=num)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [86]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 172ms/step - accuracy: 0.3977 - loss: 1.7362 - val_accuracy: 0.8800 - val_loss: 0.3691
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8971 - loss: 0.3172 - val_accuracy: 0.8950 - val_loss: 0.2594
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9293 - loss: 0.2213 - val_accuracy: 0.9300 - val_loss: 0.1826
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9779 - loss: 0.0737 - val_accuracy: 0.9450 - val_loss: 0.1547
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9889 - loss: 0.0371 - val_accuracy: 0.9650 - val_loss: 0.1471
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9912 - loss: 0.0229 - val_accuracy: 0.9350 - val_loss: 0.2103
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9886 - loss: 0.0332 - val_accuracy: 0.9500 - val_loss: 0.1831
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9864 - loss: 0.0418 - val_accuracy: 0.9700 - val_lo

In [87]:
print("Test accuracy:", test_acc)

Test accuracy: 0.9800000190734863
